# **1. Data Engineering**
### 1.1. Data load 및 description
        1.1.1. ncaa_stat
        1.1.2. college_info
        1.1.3. nba_drafted
        1.1.4. nba_undrafted
        1.1.5. Career_length
### 1.2. data merge 및 description
        1.2.1. draft_combine + CareerLength(drafted, undrafted)
        1.2.1. + ncaa_stat
### 1.3. 결측치 추정
        1.3.1. 데이터 학습
        1.3.2. 결측치 대체
        1.3.3. 평균
            1.3.3.1 재학기간평균_가중치부여X
            1.3.3.2 재학기간평균_가중치부여o(학년별)
### 1.4. EDA


### 1.1. Data load & Description<br>
    1.1.1. ncaa_stat



In [ ]:
#1.1. ncaa_stat
#1.1.1. data load
import os
import re
import numpy as np
import pandas as pd

# 파일 경로
folder_path_ncaa = r"C:\"

# 해당 폴더에 있는 모든 파일의 이름을 리스트에 저장
file_list_ncaa = os.listdir(folder_path_ncaa)

# 각 파일을 각각 읽어들여 병합
data_frames = []  # 각 파일에서 읽은 데이터프레임을 저장할 리스트
for file in file_list_ncaa:
    if file.endswith('.csv'):  # CSV 파일만 읽어오기
        file_path = os.path.join(folder_path_ncaa, file)
        try:
            temp_data = pd.read_csv(file_path, encoding='latin1')  # 데이터 로드, 인코딩을 'latin1'으로 설정
            data_frames.append(temp_data)  # 읽은 데이터프레임을 리스트에 추가
        except Exception as e:
            print(f'Error occurred while processing {file}: {e}')
            continue

# 모든 데이터프레임을 병합
ncaa_stat = pd.concat(data_frames, ignore_index=True)

# 병합 결과 확인
print(ncaa_stat.head())

In [ ]:
# #ncaa_stat 복사본 생성
# ncaa_stat_new = ncaa_stat.copy()

# 컬럼 순서 변경
# 컬럼 순서 변경
# 컬럼 순서 변경
# 컬럼 순서 변경




# 원래의 컬럼 순서를 복사
new_order = ncaa_stat.columns.tolist()

# 각 컬럼 위치 변경
new_order.insert(new_order.index('Player') + 1, new_order.pop(new_order.index('#')))
new_order.insert(new_order.index('Height') + 1, new_order.pop(new_order.index('Weight')))
new_order.insert(new_order.index('School') + 1, new_order.pop(new_order.index('Hometown')))
new_order.insert(new_order.index('Hometown') + 1, new_order.pop(new_order.index('High School')))
new_order.insert(new_order.index('Summary') + 1, new_order.pop(new_order.index('RSCI Top 100')))

# 새로운 컬럼 순서를 데이터프레임에 적용
ncaa_stat = ncaa_stat[new_order]

# 결과 확인
print(ncaa_stat.tail())



In [ ]:
#결측치 check
#결측치 check
#결측치 check
#결측치 check



missing_counts = ncaa_stat.isnull().sum()
missing_ratios = (missing_counts / len(ncaa_stat)) * 100
print(missing_ratios)

In [ ]:
#불필요한 컬럼 삭제처리: 중복되는 값, 크롤링 과정에서의 불필요한 정보 등
#불필요한 컬럼 삭제처리: 중복되는 값, 크롤링 과정에서의 불필요한 정보 등
#불필요한 컬럼 삭제처리: 중복되는 값, 크롤링 과정에서의 불필요한 정보 등
#불필요한 컬럼 삭제처리: 중복되는 값, 크롤링 과정에서의 불필요한 정보 등




ncaa_stat = ncaa_stat.drop(["Rk_per_game", "Rk_totals", "Rk_per_min", "Rk_per_poss", "RSCI Top 100",
                                    "Rk_advanced", "High School", "Hometown", "#", "Summary", 'GS_totals',
                                    'G_per_game', 'GS_per_game', 'MP_per_game', 'FG_per_game', 'FGA_per_game', 
                                    'FG%_per_game', '2P_per_game', '2PA_per_game', '2P%_per_game', '3P_per_game', 
                                    '3PA_per_game', '3P%_per_game', 'FT_per_game', 'FTA_per_game', 'FT%_per_game', 
                                    'ORB_per_game', 'DRB_per_game','TRB_per_game', 'AST_per_game', 'STL_per_game', 
                                    'BLK_per_game', 'TOV_per_game', 'PF_per_game', 'PTS_per_game', "G_per_min", 
                                    "GS_per_min", 'MP_per_min', 'FG%_per_min', '2P%_per_min', '3P%_per_min', 'FT%_per_min',
                                    "G_advanced", "GS_advanced",'MP_advanced', "_advanced", "G_per_poss", "GS_per_poss", 
                                    'MP_per_poss', "_per_poss", 'FG%_per_poss', '2P%_per_poss', '3P%_per_poss','FT%_per_poss'], axis=1)




In [ ]:
#주요 전처리1_학교명 mapping
#주요 전처리1_학교명 mapping
#주요 전처리1_학교명 mapping
#주요 전처리1_학교명 mapping




#학교명 형식 수정.
ncaa_stat['School'] = ncaa_stat['School'].str.replace('-', '_')

In [ ]:
#기본 전처리1_구분자 생성
#기본 전처리1_구분자 생성
#기본 전처리1_구분자 생성
#기본 전처리1_구분자 생성




#선수별 ID생성: 선수이름+학교

ncaa_stat['Player_ID'] = ncaa_stat['Player'] + '_' + ncaa_stat['School']
ncaa_stat['Player_ID'] = ncaa_stat.apply(lambda row: row['Player_ID'] + ' _None' if pd.isna(row['School']) else row['Player_ID'], axis=1)
player_id_col = ncaa_stat.pop('Player_ID')
ncaa_stat.insert(0, 'Player_ID', player_id_col)

In [ ]:
#기본 전처리2_특정 컬럼 전처리
#기본 전처리2_특정 컬럼 전처리
#기본 전처리2_특정 컬럼 전처리
#기본 전처리2_특정 컬럼 전처리




#3P%_totals 결측치 0 대체(3점슛을 한 번도 시도하지 않은 선수가 결측치로 되어 있음)
ncaa_stat['3P%_totals'].fillna(0, inplace=True)

In [ ]:
#기본 전처리3_단위 변경
#기본 전처리3_단위 변경
#기본 전처리3_단위 변경
#기본 전처리3_단위 변경





#'Height'컬럼 '피트-인치' --> '인치'로 변환
import re

# import pandas as pd

# "Height" 컬럼을 문자열로 변환
ncaa_stat['Height'] = ncaa_stat['Height'].astype(str)

# "0-0" 또는 "00-00" 값을 결측값으로 처리
ncaa_stat['Height'] = ncaa_stat['Height'].apply(lambda x: None if x in ['0-0', '00-00'] else x)

# 피트와 인치를 인치 단위로 변환하는 함수
def height_to_inches(height_str):
    
    if height_str is None:  # None 값인 경우 그대로 반환
        return None
        
    # 이미 숫자로 표현된 경우 그대로 반환
    if re.match(r'^\d+(\.\d+)?$', height_str):
        return float(height_str)
    
    # 피트와 인치를 추출
    match = re.match(r'(\d+)[-](\d+)', height_str)
    if match:
        feet = float(match.group(1))
        inches = float(match.group(2))
        if len(match.group(1)) == 1:  # 피트가 한 자릿수인 경우 '0'을 추가하여 두 자릿수로 변환
            feet = float('0' + match.group(1))
        return feet * 12 + inches
    
    # 매칭되지 않는 경우 NaN 반환
    return float('9999999999999')


# "Height_inch" 컬럼에 변환 결과 저장
ncaa_stat['Height_inch'] = ncaa_stat['Height'].apply(height_to_inches)

# "Height_inch" 컬럼을 "Height" 컬럼 다음에 위치시킴
ncaa_stat = ncaa_stat[['Height'] + [col for col in ncaa_stat.columns if col != 'Height']]

# 결과 확인
print(ncaa_stat[['Height', 'Height_inch']].head())


In [ ]:
#불필요 컬럼 삭제 및 순서 변경
#불필요 컬럼 삭제 및 순서 변경
#불필요 컬럼 삭제 및 순서 변경
#불필요 컬럼 삭제 및 순서 변경



#불필요한 컬럼 삭제처리 및 컬럼 순서 변경처리

ncaa_stat = ncaa_stat.drop(['Height'], axis=1)

new_order = ['Player_ID', 'Season', 'School', 'Player', 'Class', 'Pos', 'Height_inch', 'Weight',
              'G_totals', 'MP_totals', 'FG_totals', 'FGA_totals', 'FG%_totals', '2P_totals', 
              '2PA_totals', '2P%_totals', '3P_totals', '3PA_totals', '3P%_totals', 'FT_totals', 'FTA_totals', 
              'FT%_totals', 'ORB_totals', 'DRB_totals','TRB_totals', 'AST_totals', 'STL_totals', 'BLK_totals', 
              'TOV_totals', 'PF_totals', 'PTS_totals', 
              'FG_per_min', 'FGA_per_min',  '2P_per_min', '2PA_per_min',  
              '3P_per_min', '3PA_per_min',  'FT_per_min', 'FTA_per_min',  
              'TRB_per_min', 'AST_per_min', 'STL_per_min', 'BLK_per_min', 'TOV_per_min', 'PF_per_min', 
              'PTS_per_min', 
              'TS%_advanced', 'eFG%_advanced', '3PAr_advanced', 'FTr_advanced', 'PProd_advanced', 
              'ORB%_advanced', 'DRB%_advanced', 'TRB%_advanced', 'AST%_advanced', 'OBPM_advanced', 
              'DBPM_advanced', 'BPM_advanced', 'STL%_advanced', 'BLK%_advanced', 'TOV%_advanced', 
              'USG%_advanced',  'OWS_advanced', 'DWS_advanced', 'WS_advanced', 'WS/40_advanced', 'PER_advanced',
               'FG_per_poss', 'FGA_per_poss', '2P_per_poss', '2PA_per_poss', 
              '3P_per_poss', '3PA_per_poss', 'FT_per_poss', 'FTA_per_poss',  
              'TRB_per_poss', 'AST_per_poss', 'STL_per_poss', 'BLK_per_poss', 'TOV_per_poss', 'PF_per_poss', 
              'PTS_per_poss', 'ORtg_per_poss', 'DRtg_per_poss'
              ]  # 변경하고 싶은 순서대로 컬럼명을 리스트에 저장
ncaa_stat = ncaa_stat[new_order]  # 데이터프레임의 컬럼 순서를 변경


### 1.1. Data load
        1.1.2. college_info

In [ ]:
#1.2. college_info
#1.2.1. data load
#Load DATA

# import os
# import re
# import pandas as pd

# 파일 경로
file_path = "C:\\"

# 엑셀 파일 읽어오기
college_mapping = pd.read_excel(file_path)

# 데이터프레임의 처음 몇 행을 출력하여 확인
print(college_mapping.head())

In [ ]:
#컬럼명 변경 및 불필요 컬럼 삭제
#컬럼명 변경 및 불필요 컬럼 삭제
#컬럼명 변경 및 불필요 컬럼 삭제
#컬럼명 변경 및 불필요 컬럼 삭제



college_mapping = college_mapping.rename(columns={"Sports_reference(NBA)_playerinfo": "NBA"})
college_mapping = college_mapping.rename(columns={"Sports_reference(NCAA)_tableurl": "NCAA"})
college_mapping = college_mapping.drop(college_mapping.columns[2], axis=1)
college_mapping

### 1.1. Data load
        1.1.3. nba_drafted

In [ ]:
#1.4. Career_length
#1.4.1. data load

# import os
# import re
# import pandas as pd

# 파일 경로
file_path = "C:\\"

# 엑셀 파일 읽어오기
CareerLength_drafted = pd.read_csv(file_path, encoding='CP949')

# 데이터프레임의 처음 몇 행을 출력하여 확인
print(CareerLength_drafted.head())

In [ ]:
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.


CareerLength_drafted = CareerLength_drafted.drop(["Height_inch", "Twitter", "Instagram", "Shoots", "Weight_lb", 
                                   "Age", "High School"], axis=1)


# Born 결측값 선수 (Robert Vaden, Ahmad Nivins) 웹에서 생년월일 찾아서 입력

# # Born 컬럼의 결측값이 존재하는 샘플 추출
# missing_born_samples = CareerLength_drafted[CareerLength_drafted['Born'].isna()]

# Robert Vaden의 Born 컬럼 값을 March 3, 1985로 대체
CareerLength_drafted.loc[(CareerLength_drafted['Name'] == 'Robert Vaden') & (CareerLength_drafted['Born'].isna()), 'Born'] = "March 3, 1985"

# Ahmad Nivins의 Born 컬럼 값을 February 10, 1987로 대체
CareerLength_drafted.loc[(CareerLength_drafted['Name'] == 'Ahmad Nivins') & (CareerLength_drafted['Born'].isna()), 'Born'] = "February 10, 1987"


# 'Terrico White'인 샘플의 'Born' 컬럼 값을 "March 7, 1990"로 변경  (결측값은 아니었지만 날짜 형식에 차이가 있어 변경 처리)
CareerLength_drafted.loc[CareerLength_drafted['Name'] == 'Terrico White', 'Born'] = "March 7, 1990"

# 결과 확인
print(CareerLength_drafted[CareerLength_drafted['Name'].isin(['Robert Vaden', 'Ahmad Nivins', 'Terrico White'])])

In [ ]:
# 컬럼명 변경
# 컬럼명 변경
# 컬럼명 변경
# 컬럼명 변경


CareerLength_drafted.rename(columns={'NBA Debut': 'Debut'}, inplace=True)

In [ ]:
# 'Born' 컬럼의 값에 "?"가 포함된 샘플을 공백으로 변경(Tyler Harvey, Sir'Dominic Pointer, Ricky Sanchez, Satnam Singh)
# 'Born' 컬럼의 값에서 "?"를 공백으로 대체
CareerLength_drafted.loc[CareerLength_drafted['Born'].str.contains('\?', na=False), 'Born'] = CareerLength_drafted['Born'].str.replace('\?', ' ', regex=True)



In [ ]:
import pandas as pd
import re
from datetime import datetime


# 생년월일 정보만 추출하는 함수 정의
def extract_birth_date(born_str):
    # 정규 표현식을 사용하여 생년월일 추출
    match = re.search(r'([A-Za-z]+ \d{1,2}, \d{4})', born_str)
    if match:
        return match.group(0)
    else:
        return None

# 'Birth' 컬럼 생성
CareerLength_drafted['Birth'] = CareerLength_drafted['Born'].apply(extract_birth_date)


# 'Debut' 컬럼을 datetime 형식으로 변환
CareerLength_drafted['Debut'] = pd.to_datetime(CareerLength_drafted['Debut'], format='%d-%b-%y', errors='coerce')

In [ ]:
# 'Birth' 컬럼을 datetime 형식으로 변환
CareerLength_drafted['Birth'] = pd.to_datetime(CareerLength_drafted['Birth'], format='%B %d, %Y', errors='coerce')

# Draft 당시의 나이를 계산하는 함수 정의
def calculate_draft_age(row):
    if pd.notnull(row['Birth']) and pd.notnull(row['Draft_year']):
        birth_year = row['Birth'].year
        draft_year = int(row['Draft_year'])
        return draft_year - birth_year
    else:
        return None

# 'Draft_age' 컬럼 생성
CareerLength_drafted['Age'] = CareerLength_drafted.apply(calculate_draft_age, axis=1)

In [ ]:
#컬럼 별 결측치 비율 확인
#컬럼 별 결측치 비율 확인
#컬럼 별 결측치 비율 확인
#컬럼 별 결측치 비율 확인





missing_counts = CareerLength_drafted.isnull().sum()
missing_ratios = (missing_counts / len(CareerLength_drafted)) * 100
print(missing_ratios)

In [ ]:
##college변수에 값이 복수일 경우 split(college1,2,3,4) 및 ncaa기준으로 학교mapping
#주요전처리1_학교이름 맵핑(nba-ncaa)
#주요전처리1_학교이름 맵핑(nba-ncaa)
#주요전처리1_학교이름 맵핑(nba-ncaa)
#주요전처리1_학교이름 맵핑(nba-ncaa)




CareerLength_drafted.insert(6, 'College_ncaa_1', CareerLength_drafted['College_1'].map(college_mapping.set_index('NBA')['NCAA']))
CareerLength_drafted.insert(8, 'College_ncaa_2', CareerLength_drafted['College_2'].map(college_mapping.set_index('NBA')['NCAA']))
CareerLength_drafted.insert(10, 'College_ncaa_3', CareerLength_drafted['College_3'].map(college_mapping.set_index('NBA')['NCAA']))
CareerLength_drafted.insert(12, 'College_ncaa_4', CareerLength_drafted['College_4'].map(college_mapping.set_index('NBA')['NCAA']))

In [ ]:
#target variable 형식 수정.
#target variable 형식 수정.
#target variable 형식 수정.
#target variable 형식 수정.


CareerLength_drafted['Experience'] = CareerLength_drafted['Experience'].str.replace(' years', '').str.replace(' year', '')
CareerLength_drafted['Experience'] = CareerLength_drafted['Experience'].str.replace('Rookie', '0')
CareerLength_drafted['Experience'] = CareerLength_drafted['Experience'].astype(int)

In [ ]:
#기본전처리1_"Draft_num" 컬럼에서 숫자 추출하여 새로운 컬럼 생성
#기본전처리1_"Draft_num" 컬럼에서 숫자 추출하여 새로운 컬럼 생성
#기본전처리1_"Draft_num" 컬럼에서 숫자 추출하여 새로운 컬럼 생성
#기본전처리1_"Draft_num" 컬럼에서 숫자 추출하여 새로운 컬럼 생성


# career_length['Draft_pick'] = career_length['Draft_num'].str.extract(r'(\d+)\w+ pick')[0]
CareerLength_drafted['Draft_overall'] = CareerLength_drafted['Draft_num'].str.extract(r'(\d+)\w+ overall')[0]

#Draft_overall 형식 수정.
CareerLength_drafted['Draft_overall'] = CareerLength_drafted['Draft_overall'].astype(int)

# 결과 출력
print(CareerLength_drafted[['Draft_overall']])

In [ ]:
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.



# CareerLength_drafted = CareerLength_drafted.drop(["Draft_num"], axis=1)

#기본전처리2_단위 삭제 처리.
#기본전처리2_단위 삭제 처리.
#기본전처리2_단위 삭제 처리.
#기본전처리2_단위 삭제 처리.



# 'Weight_kg' 컬럼에서 'kg'를 제거합니다.
CareerLength_drafted['Weight_kg'] = CareerLength_drafted['Weight_kg'].str.replace('kg', '').astype(float)
# 'Height_cm' 컬럼에서 'cm'를 제거합니다.
CareerLength_drafted['Height_cm'] = CareerLength_drafted['Height_cm'].str.replace('cm', '').astype(float)

In [ ]:
#새로운 컬럼 생성(Debut_yrs: 데뷔날짜에서 데뷔연도만 추출)


CareerLength_drafted['Debut'] = CareerLength_drafted['Debut'].astype(str)
CareerLength_drafted['Debut_yrs'] = CareerLength_drafted['Debut'].str[:4]

In [ ]:
#컬럼 순서 변경처리
#컬럼 순서 변경처리
#컬럼 순서 변경처리
#컬럼 순서 변경처리



new_order_CareerLength_drafted = ['Name', 'Position', 'Born', 'Birth', 'Age', 'Height_cm', 'Weight_kg', 'College_1',
       'College_ncaa_1', 'College_2', 'College_ncaa_2', 'College_3',
       'College_ncaa_3', 'College_4', 'College_ncaa_4','Recruiting Rank', 'Draft_team',
       'Draft_year', 'Draft_overall', 'Debut', 'Debut_yrs', 'Experience'
              ]  # 변경하고 싶은 순서대로 컬럼명을 리스트에 저장
CareerLength_drafted = CareerLength_drafted[new_order_CareerLength_drafted]  # 데이터프레임의 컬럼 순서를 변경


In [ ]:
#Draft_overall 형식 수정.
#Draft_overall 형식 수정.
#Draft_overall 형식 수정.
#Draft_overall 형식 수정.


CareerLength_drafted['Draft_overall'] = CareerLength_drafted['Draft_overall'].astype(int)

In [ ]:
#주요전처리2_선수이름 맵핑
#주요전처리2_선수이름 맵핑
#주요전처리2_선수이름 맵핑
#주요전처리2_선수이름 맵핑




# 1. Frank Mason III(nba형식/2017darft/Kansas): Frank Mason(NCAA형식)-->Frank Mason으로 맵핑
CareerLength_drafted['Name'].replace({'Frank Mason III': 'Frank Mason'}, inplace=True)

# 2. Glen Rice Jr.(nba형식/2013draft/georgia-tech): Glen Rice (NCAA형식) --> Glen Rice로 맵핑
CareerLength_drafted['Name'].replace({'Glen Rice Jr.': 'Glen Rice'}, inplace=True)

# 4. James Ennis III(nba형식/2013draft/long-beach-state): James Ennis(NCAA형식) -->James Ennis로 맵핑
CareerLength_drafted['Name'].replace({'James Ennis III': 'James Ennis'}, inplace=True)

# 5. Jeff Taylor(nba형식/2012draft/Vanderbilt): Jeffery Taylor(NCAA형식) --> Jeffery Tylor로 맵핑
CareerLength_drafted['Name'].replace({'Jeff Taylor': 'Jeffery Taylor'}, inplace=True)

# 9. Glenn Robinson lll(nba형식/2014draft/michigan): Glenn Robinson(NCAA형식) --> Glenn Robinson으로 맵핑
#Glenn Robinson(1994draft/perdue)도 있음. Glenn Robinson III(michigan)의 아버지
CareerLength_drafted['Name'].replace({'Glenn Robinson III': 'Glenn Robinson'}, inplace=True)

#1. David Young --> Dave Young:ncaa에서의 표기법으로 맞춰 줌.
# ' David Young(nba형식/2004draft/xavier, NC central)'를 'Dave Young(NCAA형식)'로 변경
# 이 선수의 경우 NC central소속 기록은 있으나 경기기록은 없음. 원인불명.
CareerLength_drafted['Name'].replace({'David Young': 'Dave Young'}, inplace=True)

#3. Patrick Ewing --> Pat Ewing:ncaa에서의 표기법으로 맞춰 줌.
# 'Patrick Ewing(nba형식/2008draft/indiana,georgetown)'를 'Pat Ewing(NCAA형식)'으로 변경
# Patrick Ewing(1985draft)의 아들(아버지는 실험대상아님)
CareerLength_drafted['Name'].replace({'Patrick Ewing': 'Pat Ewing'}, inplace=True)

#4. Wesley Johnson --> Wes Johnson:ncaa에서의 표기법으로 맞춰 줌.
# 'Wesley Johnson(nba형식,2010draft/iowa, syracuse)'을 'Wes Johnson(NCAA형식)'으로 변경
CareerLength_drafted['Name'].replace({'Wesley Johnson': 'Wes Johnson'}, inplace=True)

#5. Ahman Nivins --> Ahmad Nivins 원인파악은 못했지만 우연하게 발견. 이름 중 철자하나가 틀리게 크롤링 됨 --> 코드로 반영하지 않고 raw데이터에 직접 반영함
# CareerLength_drafted['Name'].replace({'Ahman Nivins': 'Ahmad Nivins'}, inplace=True)

#5. JP Tokoto --> J.P. Tokoto 논문 마지막 표 작성 중 발견.  
CareerLength_drafted['Name'].replace({'JP Tokoto': 'J.P. Tokoto'}, inplace=True)


In [ ]:
#특정컬럼 형식 수정
#특정컬럼 형식 수정
#특정컬럼 형식 수정
#특정컬럼 형식 수정



CareerLength_drafted['Name'] = CareerLength_drafted['Name'].astype(str)  # 'Name'을 문자열로 변환
CareerLength_drafted['College_ncaa_1'] = CareerLength_drafted['College_ncaa_1'].astype(str)  # 'School'을 문자열로 변환
CareerLength_drafted['College_ncaa_2'] = CareerLength_drafted['College_ncaa_2'].astype(str)  # 'School'을 문자열로 변환
CareerLength_drafted['College_ncaa_3'] = CareerLength_drafted['College_ncaa_3'].astype(str)  # 'School'을 문자열로 변환
CareerLength_drafted['College_ncaa_4'] = CareerLength_drafted['College_ncaa_4'].astype(str)  # 'School'을 문자열로 변환


#값 형태 수정 
#값 형태 수정 
#값 형태 수정 
#값 형태 수정 



CareerLength_drafted['College_ncaa_1'] = CareerLength_drafted['College_ncaa_1'].str.replace('-', '_')
CareerLength_drafted['College_ncaa_2'] = CareerLength_drafted['College_ncaa_2'].str.replace('-', '_')
CareerLength_drafted['College_ncaa_3'] = CareerLength_drafted['College_ncaa_3'].str.replace('-', '_')
CareerLength_drafted['College_ncaa_4'] = CareerLength_drafted['College_ncaa_4'].str.replace('-', '_')

In [ ]:
#주요전처리3_특정선수 학교 디비전 변경처리(자세한 내용은 실험대상 정리 파일 참조)
#주요전처리3_특정선수 학교 디비전 변경처리(자세한 내용은 실험대상 정리 파일 참조)
#주요전처리3_특정선수 학교 디비전 변경처리(자세한 내용은 실험대상 정리 파일 참조)
#주요전처리3_특정선수 학교 디비전 변경처리(자세한 내용은 실험대상 정리 파일 참조)





CareerLength_drafted.loc[CareerLength_drafted['Name'] == 'Kebu Stewart', 'College_ncaa_2'] = 'not_NCAA_Division_1'   #cal-state-bakersfield
CareerLength_drafted.loc[CareerLength_drafted['Name'] == 'Jerome Beasley', 'College_ncaa_2'] = 'not_NCAA_Division_1'  #north_dakota
CareerLength_drafted.loc[CareerLength_drafted['Name'] == 'Michael McDonald', 'College_ncaa_1'] = 'not_NCAA_Division_1'   #utah-valley
CareerLength_drafted.loc[CareerLength_drafted['Name'] == 'Travis Hansen', 'College_ncaa_1'] = 'not_NCAA_Division_1'      #utah-valley
CareerLength_drafted.loc[CareerLength_drafted['Name'] == 'Andrew Betts', 'College_ncaa_1'] = 'not_NCAA_Division_1'     #long-island-university
CareerLength_drafted.loc[CareerLength_drafted['Name'] == 'Anthony Jones', 'College_ncaa_2'] = 'not_NCAA_Division_1'     #oral-roberts
CareerLength_drafted.loc[CareerLength_drafted['Name'] == 'Dave Young', 'College_ncaa_2'] = 'not_NCAA_Division_1'     #north_carolina_central




In [ ]:
#주요전처리4_선수id생성: 선수이름+학교명(복수의 학교에 소속되었던 선수들은 각각의 학교별로 id생성)
#주요전처리4_선수id생성: 선수이름+학교명(복수의 학교에 소속되었던 선수들은 각각의 학교별로 id생성)
#주요전처리4_선수id생성: 선수이름+학교명(복수의 학교에 소속되었던 선수들은 각각의 학교별로 id생성)
#주요전처리4_선수id생성: 선수이름+학교명(복수의 학교에 소속되었던 선수들은 각각의 학교별로 id생성)





CareerLength_drafted['Player_ID_1'] = CareerLength_drafted['Name'] + '_' + CareerLength_drafted['College_ncaa_1']
CareerLength_drafted['Player_ID_1'] = CareerLength_drafted.apply(lambda row: row['Player_ID_1'] + ' _None' if pd.isna(row['College_ncaa_1']) else row['Player_ID_1'], axis=1)
player_id_col = CareerLength_drafted.pop('Player_ID_1')
CareerLength_drafted.insert(0, 'Player_ID_1', player_id_col)


CareerLength_drafted['Player_ID_2'] = CareerLength_drafted['Name'] + '_' + CareerLength_drafted['College_ncaa_2']
CareerLength_drafted['Player_ID_2'] = CareerLength_drafted.apply(lambda row: row['Player_ID_2'] + ' _None' if pd.isna(row['College_ncaa_2']) else row['Player_ID_2'], axis=1)
player_id_col = CareerLength_drafted.pop('Player_ID_2')
CareerLength_drafted.insert(1, 'Player_ID_2', player_id_col)


CareerLength_drafted['Player_ID_3'] = CareerLength_drafted['Name'] + '_' + CareerLength_drafted['College_ncaa_3']
CareerLength_drafted['Player_ID_3'] = CareerLength_drafted.apply(lambda row: row['Player_ID_3'] + ' _None' if pd.isna(row['College_ncaa_3']) else row['Player_ID_3'], axis=1)
player_id_col = CareerLength_drafted.pop('Player_ID_3')
CareerLength_drafted.insert(2, 'Player_ID_3', player_id_col)



CareerLength_drafted['Player_ID_4'] = CareerLength_drafted['Name'] + '_' + CareerLength_drafted['College_ncaa_4']
CareerLength_drafted['Player_ID_4'] = CareerLength_drafted.apply(lambda row: row['Player_ID_4'] + ' _None' if pd.isna(row['College_ncaa_4']) else row['Player_ID_4'], axis=1)
player_id_col = CareerLength_drafted.pop('Player_ID_4')
CareerLength_drafted.insert(3, 'Player_ID_4', player_id_col)


In [ ]:
new_order_CareerLength_drafted = ['Player_ID_1', 'Player_ID_2', 'Player_ID_3', 'Player_ID_4','Name', 'Position', 'Born', 'Birth', 'Age', 'Height_cm', 'Weight_kg', 'College_1',
       'College_ncaa_1', 'College_2', 'College_ncaa_2', 'College_3',
       'College_ncaa_3', 'College_4', 'College_ncaa_4','Recruiting Rank', 'Draft_team',
       'Draft_year', 'Draft_overall', 'Debut', 'Experience'
              ]  # 변경하고 싶은 순서대로 컬럼명을 리스트에 저장
CareerLength_drafted = CareerLength_drafted[new_order_CareerLength_drafted]  # 데이터프레임의 컬럼 순서를 변경

### ncaa_stat과 CareerLength_drafted에서 출신학교가 동일한 동명이인 선수 'Player_ID'변경 처리

In [ ]:
#동명이인 선수 Player_ID 변경 처리
#동명이인 선수 Player_ID 변경 처리
#동명이인 선수 Player_ID 변경 처리
#동명이인 선수 Player_ID 변경 처리






#1-1 Brian Oliver(georgia_tech)
##ncaa_stat_new

# 조건에 맞는 샘플 추출
condition = (ncaa_stat['Player_ID'] == "Brian Oliver_georgia_tech") & (ncaa_stat['Season'].isin(['1986-87', '1987-88', '1988-89', '1989-90']))
samples_to_update = ncaa_stat[condition]
# 'Player_ID' 값을 변경
ncaa_stat.loc[condition, 'Player_ID'] = "Brian Oliver_georgia_tech_1990"


In [ ]:
#1-2 Brian Oliver(georgia_tech)
##Careerlength_drafted
# 조건에 맞는 샘플 추출
condition = CareerLength_drafted['Player_ID_1'] == 'Brian Oliver_georgia_tech'
# 'Player_ID_1' 값을 변경
CareerLength_drafted.loc[condition, 'Player_ID_1'] = 'Brian Oliver_georgia_tech_1990'

In [ ]:
#2-1 Gary Payton(oregon_state)
##ncaa_stat
# 조건에 맞는 샘플 추출
condition = (ncaa_stat['Player_ID'] == "Gary Payton_oregon_state") & (ncaa_stat['Season'].isin(['1986-87', '1987-88', '1988-89', '1989-90']))
samples_to_update = ncaa_stat[condition]
# 'Player_ID' 값을 변경
ncaa_stat.loc[condition, 'Player_ID'] = "Gary Payton_oregon_state_1990"


In [ ]:
#2-2 Gary Payton(oregon_state)
##Careerlength_drafted
# 조건에 맞는 샘플 추출
condition = CareerLength_drafted['Player_ID_1'] == 'Gary Payton_oregon_state'
# 'Player_ID_1' 값을 변경
CareerLength_drafted.loc[condition, 'Player_ID_1'] = 'Gary Payton_oregon_state_1990'

In [ ]:
#3-1 Steve Rogers(alabama_state)
##ncaa_stat
# 조건에 맞는 샘플 추출
condition = (ncaa_stat['Player_ID'] == "Steve Rogers_alabama_state") & (ncaa_stat['Season'].isin(['1989-90', '1990-91', '1991-92']))
samples_to_update = ncaa_stat[condition]
# 'Player_ID' 값을 변경
ncaa_stat.loc[condition, 'Player_ID'] = "Steve Rogers_alabama_state_1992"

In [ ]:
#3-2 Steve Rogers(alabama_state)
##Careerlength_drafted
# 조건에 맞는 샘플 추출
condition = CareerLength_drafted['Player_ID_2'] == 'Steve Rogers_alabama_state'
# 'Player_ID_1' 값을 변경
CareerLength_drafted.loc[condition, 'Player_ID_2'] = 'Steve Rogers_alabama_state_1992'

In [ ]:
#4-1 Chris Corchiani(north_carolina_state)
##ncaa_stat
# 조건에 맞는 샘플 추출
condition = (ncaa_stat['Player_ID'] == "Chris Corchiani_north_carolina_state") & (ncaa_stat['Season'].isin(['1987-88', '1988-89', '1989-90', '1990-91']))
samples_to_update = ncaa_stat[condition]
# 'Player_ID' 값을 변경
ncaa_stat.loc[condition, 'Player_ID'] = "Chris Corchiani_north_carolina_state_1991"

In [ ]:
#4-2 Chris Corchiani(north_carolina_state)
##Careerlength_drafted
# 조건에 맞는 샘플 추출
condition = CareerLength_drafted['Player_ID_1'] == 'Chris Corchiani_north_carolina_state'
# 'Player_ID_1' 값을 변경
CareerLength_drafted.loc[condition, 'Player_ID_1'] = 'Chris Corchiani_north_carolina_state_1991'

In [ ]:
#5-1 Chris Herren(boston_college)
##ncaa_stat
# 조건에 맞는 샘플 추출
condition = (ncaa_stat['Player_ID'] == "Chris Herren_boston_college") & (ncaa_stat['Season'].isin(['1994-95']))
samples_to_update = ncaa_stat[condition]
# 'Player_ID' 값을 변경
ncaa_stat.loc[condition, 'Player_ID'] = "Chris Herren_boston_college_1999"

In [ ]:
#5-2 Chris Herren(boston_college)
##Careerlength_drafted
# 조건에 맞는 샘플 추출
condition = CareerLength_drafted['Player_ID_1'] == 'Chris Herren_boston_college'
# 'Player_ID_1' 값을 변경
CareerLength_drafted.loc[condition, 'Player_ID_1'] = 'Chris Herren_boston_college_1999'

### 1.1. Data load
        1.1.4. nba_undrafted

In [ ]:
# 파일 경로
file_path = "C:\\"

# 엑셀 파일 읽어오기
CareerLength_undrafted = pd.read_csv(file_path, encoding='CP949')

# 데이터프레임의 처음 몇 행을 출력하여 확인
print(CareerLength_undrafted.head())

In [ ]:
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.
#불필요한 컬럼 삭제 처리.





CareerLength_undrafted = CareerLength_undrafted.drop(["Height_inch", "Twitter", "Instagram", "Shoots", "Weight_lb", "Age",
                                   "High School"], axis=1)

In [ ]:
# 컬럼명 변경
# 컬럼명 변경
# 컬럼명 변경
# 컬럼명 변경


CareerLength_undrafted.rename(columns={'NBA Debut': 'Debut'}, inplace=True)

In [ ]:
import pandas as pd
import re
from datetime import datetime

# # 샘플 데이터프레임 생성
# data = {
#     'Name': ['Player1', 'Player2', 'Player3'],
#     'Born': ['June 21, 1967 in Mobile, Alabama us', 'September 5, 1968 in Hagerstown, Maryland us', 'March 19, 1968 in Cincinnati, Ohio us']
# }

# CareerLength_drafted = pd.DataFrame(data)


##이미 drafted에서 함수를 구현했으므로 주석 처리함.
##이미 drafted에서 함수를 구현했으므로 주석 처리함.
##이미 drafted에서 함수를 구현했으므로 주석 처리함.
##이미 drafted에서 함수를 구현했으므로 주석 처리함.



# # 생년월일 정보만 추출하는 함수 정의
# def extract_birth_date(born_str):
#     # 정규 표현식을 사용하여 생년월일 추출
#     match = re.search(r'([A-Za-z]+ \d{1,2}, \d{4})', born_str)
#     if match:
#         return match.group(0)
#     else:
#         return None

# 'Birth' 컬럼 생성
CareerLength_undrafted['Birth'] = CareerLength_undrafted['Born'].apply(extract_birth_date)


In [ ]:
# 'Birth' 컬럼을 datetime 형식으로 변환
CareerLength_undrafted['Birth'] = pd.to_datetime(CareerLength_undrafted['Birth'], format='%B %d, %Y', errors='coerce')

# 'Debut' 컬럼을 datetime 형식으로 변환
CareerLength_undrafted['Debut'] = pd.to_datetime(CareerLength_undrafted['Debut'], format='%d-%b-%y', errors='coerce')

# Draft 당시의 나이를 계산하는 함수 정의 (언드래프트의 경우 데뷔날짜로)
def calculate_draft_age(row):
    if pd.notnull(row['Birth']) and pd.notnull(row['Debut']):
        birth_year = row['Birth'].year
        debut_year = row['Debut'].year
        return debut_year - birth_year
    else:
        return None

# 'Age' 컬럼 생성
CareerLength_undrafted['Age'] = CareerLength_undrafted.apply(calculate_draft_age, axis=1)

# 결과 확인
print(CareerLength_undrafted)


In [ ]:
#컬럼 별 결측치 비율 확인
#컬럼 별 결측치 비율 확인
#컬럼 별 결측치 비율 확인
#컬럼 별 결측치 비율 확인





missing_counts = CareerLength_undrafted.isnull().sum()
missing_ratios = (missing_counts / len(CareerLength_undrafted)) * 100
print(missing_ratios)

In [ ]:
##college변수에 값이 복수일 경우 split(college1,2,3,4) 및 ncaa기준으로 학교mapping
#주요전처리1_학교이름 맵핑(nba-ncaa)
#주요전처리1_학교이름 맵핑(nba-ncaa)
#주요전처리1_학교이름 맵핑(nba-ncaa)
#주요전처리1_학교이름 맵핑(nba-ncaa)




CareerLength_undrafted.insert(6, 'College_ncaa_1', CareerLength_undrafted['College_1'].map(college_mapping.set_index('NBA')['NCAA']))
CareerLength_undrafted.insert(8, 'College_ncaa_2', CareerLength_undrafted['College_2'].map(college_mapping.set_index('NBA')['NCAA']))
CareerLength_undrafted.insert(10, 'College_ncaa_3', CareerLength_undrafted['College_3'].map(college_mapping.set_index('NBA')['NCAA']))
CareerLength_undrafted.insert(12, 'College_ncaa_4', CareerLength_undrafted['College_4'].map(college_mapping.set_index('NBA')['NCAA']))

In [ ]:
#target variable 형식 수정.
#target variable 형식 수정.
#target variable 형식 수정.
#target variable 형식 수정.



CareerLength_undrafted['Experience'] = CareerLength_undrafted['Experience'].str.replace(' years', '').str.replace(' year', '')
CareerLength_undrafted['Experience'] = CareerLength_undrafted['Experience'].str.replace('Rookie', '0')
CareerLength_undrafted['Experience'] = CareerLength_undrafted['Experience'].astype(int)

In [ ]:
#기본전처리1_"Draft_num" 컬럼에서 숫자 추출하여 새로운 컬럼 생성
#기본전처리1_"Draft_num" 컬럼에서 숫자 추출하여 새로운 컬럼 생성
#기본전처리1_"Draft_num" 컬럼에서 숫자 추출하여 새로운 컬럼 생성
#기본전처리1_"Draft_num" 컬럼에서 숫자 추출하여 새로운 컬럼 생성


# # career_length['Draft_pick'] = career_length['Draft_num'].str.extract(r'(\d+)\w+ pick')[0]
# CareerLength_undrafted['Draft_overall'] = CareerLength_undrafted['Draft_num'].str.extract(r'(\d+)\w+ overall')[0]


#CareerLength_undrafted["Draft_year"]가 null값이므로 "undrafted"로 채워줌
CareerLength_undrafted['Draft_overall'] = 'undrafted'



#Draft_overall 형식 수정.
# CareerLength_undrafted['Draft_overall'] = CareerLength_undrafted['Draft_overall'].astype(int)

# 결과 출력
print(CareerLength_undrafted[['Draft_overall']])

In [ ]:
#주요전처리2_선수이름 맵핑(크롤링 과정에서 이름이 깨진 선수 중 실험대상에 포함되는 선수 이름 맵핑 및 변경)_상세내용은 엑셀로 정리한 실험대상확인 파일 참고.
#주요전처리2_선수이름 맵핑(크롤링 과정에서 이름이 깨진 선수 중 실험대상에 포함되는 선수 이름 맵핑 및 변경)_상세내용은 엑셀로 정리한 실험대상확인 파일 참고.
#주요전처리2_선수이름 맵핑(크롤링 과정에서 이름이 깨진 선수 중 실험대상에 포함되는 선수 이름 맵핑 및 변경)_상세내용은 엑셀로 정리한 실험대상확인 파일 참고.
#주요전처리2_선수이름 맵핑(크롤링 과정에서 이름이 깨진 선수 중 실험대상에 포함되는 선수 이름 맵핑 및 변경)_상세내용은 엑셀로 정리한 실험대상확인 파일 참고.





# 1. 
CareerLength_undrafted['Name'].replace({'Jorge Guti챕rrez': 'Jorge Gutierrez'}, inplace=True)

# 2. 
CareerLength_undrafted['Name'].replace({'Pepe S찼nchez': 'Pepe Sanchez'}, inplace=True)

# 3. 
CareerLength_undrafted['Name'].replace({'Rub챕n Garc챕s': 'Ruben Garces'}, inplace=True)

# 4. 
CareerLength_undrafted['Name'].replace({'힋ar큰nas Jasikevi훾ius': 'Sarunas Jasikevicius'}, inplace=True)

# 5. 
CareerLength_undrafted['Name'].replace({'Predrag Savovi훶': 'Predrag Savovic'}, inplace=True)

#6. 
CareerLength_undrafted['Name'].replace({'횁ngel Delgado': 'Angel Delgado'}, inplace=True)

#7.
CareerLength_undrafted['Name'].replace({'Kiwane Lemorris Garris': 'Kiwane Garris'}, inplace=True)

#8.
CareerLength_undrafted['Name'].replace({'John Lucas III': 'John Lucas'}, inplace=True)

#9.
CareerLength_undrafted['Name'].replace({'Lou Amundson': 'Louis Amundson'}, inplace=True)

#10.
CareerLength_undrafted['Name'].replace({'Eugene Jeter': 'Pooh Jeter'}, inplace=True)

#11.
CareerLength_undrafted['Name'].replace({'Larry Drew II': 'Larry Drew'}, inplace=True)

#12.
CareerLength_undrafted['Name'].replace({'Sheldon Mac': 'Sheldon McClellan'}, inplace=True)

#13.
CareerLength_undrafted['Name'].replace({'Naz Mitrou-Long': 'Nazareth Mitrou-Long'}, inplace=True)

#14.
CareerLength_undrafted['Name'].replace({'James Webb III': 'James Webb'}, inplace=True)

#15.
CareerLength_undrafted['Name'].replace({'Walt Lemon Jr.': 'Walt Lemon'}, inplace=True)

In [ ]:
#주요전처리3_특정선수 학교 디비전 변경처리(자세한 내용은 실험대상 정리 파일 참조)
#주요전처리3_특정선수 학교 디비전 변경처리(자세한 내용은 실험대상 정리 파일 참조)
#주요전처리3_특정선수 학교 디비전 변경처리(자세한 내용은 실험대상 정리 파일 참조)
#주요전처리3_특정선수 학교 디비전 변경처리(자세한 내용은 실험대상 정리 파일 참조)

CareerLength_undrafted.loc[CareerLength_undrafted['Name'] == 'Horacio Llamas', 'College_ncaa_1'] = 'not_NCAA_Division_1'   #grand-canyon
CareerLength_undrafted.loc[CareerLength_undrafted['Name'] == 'Brett Szabo', 'College_ncaa_1'] = 'not_NCAA_Division_1'  #augustana-il
CareerLength_undrafted.loc[CareerLength_undrafted['Name'] == 'Garret Siler', 'College_ncaa_1'] = 'not_NCAA_Division_1'   #augusta-state
CareerLength_undrafted.loc[CareerLength_undrafted['Name'] == 'Freddie Gillespie', 'College_ncaa_1'] = 'not_NCAA_Division_1'   #carleton-college
CareerLength_undrafted.loc[CareerLength_undrafted['Name'] == 'Thomas Hamilton', 'College_ncaa_1'] = 'nan'   #pitt 대학 진학했으나 농구안했으므로 nan처리




In [ ]:
#특정컬럼 형식 수정
#특정컬럼 형식 수정
#특정컬럼 형식 수정
#특정컬럼 형식 수정



CareerLength_undrafted['Name'] = CareerLength_undrafted['Name'].astype(str)  # 'Name'을 문자열로 변환
CareerLength_undrafted['College_ncaa_1'] = CareerLength_undrafted['College_ncaa_1'].astype(str)  # 'School'을 문자열로 변환
CareerLength_undrafted['College_ncaa_2'] = CareerLength_undrafted['College_ncaa_2'].astype(str)  # 'School'을 문자열로 변환
CareerLength_undrafted['College_ncaa_3'] = CareerLength_undrafted['College_ncaa_3'].astype(str)  # 'School'을 문자열로 변환
CareerLength_undrafted['College_ncaa_4'] = CareerLength_undrafted['College_ncaa_4'].astype(str)  # 'School'을 문자열로 변환


#값 형태 수정 
#값 형태 수정 
#값 형태 수정 
#값 형태 수정 



CareerLength_undrafted['College_ncaa_1'] = CareerLength_undrafted['College_ncaa_1'].str.replace('-', '_')
CareerLength_undrafted['College_ncaa_2'] = CareerLength_undrafted['College_ncaa_2'].str.replace('-', '_')
CareerLength_undrafted['College_ncaa_3'] = CareerLength_undrafted['College_ncaa_3'].str.replace('-', '_')
CareerLength_undrafted['College_ncaa_4'] = CareerLength_undrafted['College_ncaa_4'].str.replace('-', '_')

In [ ]:
#주요전처리4_선수id생성: 선수이름+학교명(복수의 학교에 소속되었던 선수들은 각각의 학교별로 id생성)
#주요전처리4_선수id생성: 선수이름+학교명(복수의 학교에 소속되었던 선수들은 각각의 학교별로 id생성)
#주요전처리4_선수id생성: 선수이름+학교명(복수의 학교에 소속되었던 선수들은 각각의 학교별로 id생성)
#주요전처리4_선수id생성: 선수이름+학교명(복수의 학교에 소속되었던 선수들은 각각의 학교별로 id생성)





CareerLength_undrafted['Player_ID_1'] = CareerLength_undrafted['Name'] + '_' + CareerLength_undrafted['College_ncaa_1']
CareerLength_undrafted['Player_ID_1'] = CareerLength_undrafted.apply(lambda row: row['Player_ID_1'] + ' _None' if pd.isna(row['College_ncaa_1']) else row['Player_ID_1'], axis=1)
player_id_col = CareerLength_undrafted.pop('Player_ID_1')
CareerLength_undrafted.insert(0, 'Player_ID_1', player_id_col)


CareerLength_undrafted['Player_ID_2'] = CareerLength_undrafted['Name'] + '_' + CareerLength_undrafted['College_ncaa_2']
CareerLength_undrafted['Player_ID_2'] = CareerLength_undrafted.apply(lambda row: row['Player_ID_2'] + ' _None' if pd.isna(row['College_ncaa_2']) else row['Player_ID_2'], axis=1)
player_id_col = CareerLength_undrafted.pop('Player_ID_2')
CareerLength_undrafted.insert(1, 'Player_ID_2', player_id_col)


CareerLength_undrafted['Player_ID_3'] = CareerLength_undrafted['Name'] + '_' + CareerLength_undrafted['College_ncaa_3']
CareerLength_undrafted['Player_ID_3'] = CareerLength_undrafted.apply(lambda row: row['Player_ID_3'] + ' _None' if pd.isna(row['College_ncaa_3']) else row['Player_ID_3'], axis=1)
player_id_col = CareerLength_undrafted.pop('Player_ID_3')
CareerLength_undrafted.insert(2, 'Player_ID_3', player_id_col)



CareerLength_undrafted['Player_ID_4'] = CareerLength_undrafted['Name'] + '_' + CareerLength_undrafted['College_ncaa_4']
CareerLength_undrafted['Player_ID_4'] = CareerLength_undrafted.apply(lambda row: row['Player_ID_4'] + ' _None' if pd.isna(row['College_ncaa_4']) else row['Player_ID_4'], axis=1)
player_id_col = CareerLength_undrafted.pop('Player_ID_4')
CareerLength_undrafted.insert(3, 'Player_ID_4', player_id_col)


In [ ]:
CareerLength_undrafted['Debut'] = CareerLength_undrafted['Debut'].astype(str)
CareerLength_undrafted['Debut_yrs'] = CareerLength_undrafted['Debut'].str[:4]

In [ ]:
#컬럼 순서 변경처리
#컬럼 순서 변경처리
#컬럼 순서 변경처리
#컬럼 순서 변경처리



new_order_CareerLength_undrafted = ['Player_ID_1', 'Player_ID_2', 'Player_ID_3', 'Player_ID_4','Name', 'Position', 'Born', 'Birth', 'Age', 'Height_cm', 'Weight_kg', 
       'College_ncaa_1',  'College_ncaa_2', 'College_ncaa_3', 'College_ncaa_4','Recruiting Rank', 'Draft_team',
       'Draft_year', 'Draft_overall', 'Debut', 'Debut_yrs', 'Experience'
              ]  # 변경하고 싶은 순서대로 컬럼명을 리스트에 저장
CareerLength_undrafted = CareerLength_undrafted[new_order_CareerLength_undrafted]  # 데이터프레임의 컬럼 순서를 변경

### ncaa_stat와 CareerLength_drafted에서 동명이인이며 출신학교가 동일한 선수 'Player_ID'변경 처리



In [ ]:
#주요전처리5_동명이인 선수 Player_ID 변경 처리
#주요전처리5_동명이인 선수 Player_ID 변경 처리
#주요전처리5_동명이인 선수 Player_ID 변경 처리
#주요전처리5_동명이인 선수 Player_ID 변경 처리





#1-1. Scott Williams(north_carolina)
##ncaa_stat

# 조건에 맞는 샘플 추출
condition = (ncaa_stat['Player_ID'] == "Scott Williams_north_carolina") & (ncaa_stat['Season'].isin(['1986-87', '1987-88', '1988-89', '1989-90']))
samples_to_update = ncaa_stat[condition]
# 'Player_ID' 값을 변경
ncaa_stat.loc[condition, 'Player_ID'] = "Scott Williams_north_carolina_1990"


In [ ]:
#1-2. Scott Williams(north_carolina)
##Careerlength_undrafted
# 조건에 맞는 샘플 추출
condition = CareerLength_undrafted['Player_ID_1'] == 'Scott Williams_north_carolina'
# 'Player_ID_1' 값을 변경
CareerLength_undrafted.loc[condition, 'Player_ID_1'] = 'Scott Williams_north_carolina_1990'

In [ ]:
#2-1. Jermaine Jackson(detroit_mercy)
##ncaa_stat

# 조건에 맞는 샘플 추출
condition = (ncaa_stat['Player_ID'] == "Jermaine Jackson_detroit_mercy") & (ncaa_stat['Season'].isin(['1995-96', '1996-97', '1997-98', '1998-99']))
samples_to_update = ncaa_stat[condition]
# 'Player_ID' 값을 변경
ncaa_stat.loc[condition, 'Player_ID'] = "Jermaine Jackson_detroit_mercy_1999"


In [ ]:
#2-2. Jermaine Jackson(detroit_mercy)
##Careerlength_undrafted
# 조건에 맞는 샘플 추출
condition = CareerLength_undrafted['Player_ID_1'] == 'Jermaine Jackson_detroit_mercy'
# 'Player_ID_1' 값을 변경
CareerLength_undrafted.loc[condition, 'Player_ID_1'] = 'Jermaine Jackson_detroit_mercy_1999'

1.5. Draft combine

In [ ]:
#Draft combine results


import os
import re
import pandas as pd

# 파일 경로
folder_path_draft_combine = r"C:\"

# 해당 폴더에 있는 모든 파일의 이름을 리스트에 저장
file_list_draft_combine = os.listdir(folder_path_draft_combine)

# 각 파일을 각각 읽어들여 병합
data_frames = []  # 각 파일에서 읽은 데이터프레임을 저장할 리스트
for file in file_list_draft_combine:
    if file.endswith('.csv'):  # CSV 파일만 읽어오기
        file_path = os.path.join(folder_path_draft_combine, file)
        try:
            temp_data = pd.read_csv(file_path, encoding='latin1')  # 데이터 로드, 인코딩을 'latin1'으로 설정
            data_frames.append(temp_data)  # 읽은 데이터프레임을 리스트에 추가
        except Exception as e:
            print(f'Error occurred while processing {file}: {e}')
            continue

# 모든 데이터프레임을 병합
draft_combine = pd.concat(data_frames, ignore_index=True)

# 병합 결과 확인
print(draft_combine.head())

In [ ]:
#Season 표기형식 수정(일부)
#Season 표기형식 수정(일부)
#Season 표기형식 수정(일부)
#Season 표기형식 수정(일부)




draft_combine.loc[draft_combine['Season'] == 'Jan-00', 'Season'] = '2000-01'
draft_combine.loc[draft_combine['Season'] == 'Feb-01', 'Season'] = '2001-02'
draft_combine.loc[draft_combine['Season'] == 'Mar-02', 'Season'] = '2002-03'
draft_combine.loc[draft_combine['Season'] == 'Apr-03', 'Season'] = '2003-04'

In [ ]:
# 연도 추출
# 연도 추출
# 연도 추출
# 연도 추출


draft_combine['combine_year'] = draft_combine['Season'].apply(lambda x: x.split('-')[0])  # 연도 추출

In [ ]:
# 특정 컬럼 값 형태 수정
# 특정 컬럼 값 형태 수정
# 특정 컬럼 값 형태 수정
# 특정 컬럼 값 형태 수정


# 'Body_fat_pct' 컬럼에서 '%'를 제거.

draft_combine['Body_fat_pct'] = draft_combine['Body_fat_pct'].str.replace('%', '').astype(object)

In [ ]:
# 모든 컬럼에 대해 '-'(값 존재x)를 NaN으로 대체.
# 모든 컬럼에 대해 '-'(값 존재x)를 NaN으로 대체.
# 모든 컬럼에 대해 '-'(값 존재x)를 NaN으로 대체.
# 모든 컬럼에 대해 '-'(값 존재x)를 NaN으로 대체.


for column in draft_combine.columns:
    draft_combine[column] = draft_combine[column].replace('-', np.nan)

In [ ]:
#특정 컬럼 '피트-인치' --> '피트'로 변환 처리.
#특정 컬럼 '피트-인치' --> '피트'로 변환 처리.
#특정 컬럼 '피트-인치' --> '피트'로 변환 처리.
#특정 컬럼 '피트-인치' --> '피트'로 변환 처리.




import re

# 데이터프레임의 컬럼을 문자열로 변환
for col in ['Wingspan_inch', 'Standing_Reach_inch', 'Height_wo_Shoes', 'Height_w_Shoes']:
    draft_combine[col] = draft_combine[col].astype(str)

# 피트와 인치를 인치 단위로 변환하는 함수
def height_to_inches(height_str):
    # 이미 숫자인 경우 그대로 반환
    if re.match(r'^\d+(\.\d+)?$', height_str):
        return float(height_str)
    
    # 피트와 인치를 추출
    match = re.match(r'(\d+)[\'"]\s*(\d+(\.\d+)?)["]?', height_str)
    if match:
        feet = float(match.group(1))
        inches = float(match.group(2))
        return feet * 12 + inches
    
    # 매칭되지 않는 경우 NaN 반환
    return float('NaN')

# 변환 함수를 데이터프레임의 해당 컬럼에 적용
for col in ['Wingspan_inch', 'Standing_Reach_inch', 'Height_wo_Shoes', 'Height_w_Shoes']:
    draft_combine[col] = draft_combine[col].apply(height_to_inches)




In [ ]:
#불필요한 컬럼 삭제처리
#불필요한 컬럼 삭제처리
#불필요한 컬럼 삭제처리
#불필요한 컬럼 삭제처리




draft_combine = draft_combine.drop(['Pos', 'Season', 'OFF_DRIBBLE_COELLEGE_BREAK_left_pct', 'OFF_DRIBBLE_COELLEGE_BREAK_right_pct',
                                        'OFF_DRIBBLE_COELLEGE_TOP_KEY_pct','OFF_DRIBBLE_FIFTEEN_BREAK_left_pct', 
                                        'OFF_DRIBBLE_FIFTEEN_BREAK_right_pct', 'OFF_DRIBBLE_FIFTEEN_TOP_KEY_pct',
                                        'ON_THE_MOVE_COLLEGE_pct','ON_THE_MOVE_FIFTEEN_pct','NBA_BREAK_left_pct',
                                        'NBA_BREAK_right_pct', 'NBA_CORNER_left_pct', 'NBA_CORNER_right_pct',
                                        'NBA_TOP_KEY_pct', 'COLLEGE_BREAK_left_pct','COLLEGE_BREAK_right_pct',
                                        'COLLEGE_CORNER_left_pct','COLLEGE_CORNER_right_pct','COLLEGE_TOP_KEY_pct',
                                        'FIFTEEN_BREAK_left_pct','FIFTEEN_BREAK_right_pct','FIFTEEN_CORNER_left_pct',
                                        'FIFTEEN_CORNER_right_pct','FIFTEEN_TOP_KEY_pct','FIFTEEN_TOP_KEY_pct'], axis=1)

In [ ]:
# 특정 컬럼 데이터 타입 변경.('Season', 'Name', 'Pos'를 제외한 모든 컬럼을 float으로)
# 특정 컬럼 데이터 타입 변경.('Season', 'Name', 'Pos'를 제외한 모든 컬럼을 float으로)
# 특정 컬럼 데이터 타입 변경.('Season', 'Name', 'Pos'를 제외한 모든 컬럼을 float으로)
# 특정 컬럼 데이터 타입 변경.('Season', 'Name', 'Pos'를 제외한 모든 컬럼을 float으로)





cols_to_convert = [col for col in draft_combine.columns if col not in ['Player_ID','Season', 'Name', 'Pos', 'combine_year']]
draft_combine[cols_to_convert] = draft_combine[cols_to_convert].astype(float)

In [ ]:
#Draft_combine 데이터셋 id생성(merged_data_all과 병합위한 처리)
#Draft_combine 데이터셋 id생성(merged_data_all과 병합위한 처리)
#Draft_combine 데이터셋 id생성(merged_data_all과 병합위한 처리)
#Draft_combine 데이터셋 id생성(merged_data_all과 병합위한 처리)



draft_combine['Player_ID'] = draft_combine['combine_year'] + '_' + draft_combine['Name']
# draft_combine['Player_ID'] = draft_combine.apply(lambda row: row['Player_ID_1'] + ' _None' if pd.isna(row['College_ncaa_1']) else row['Player_ID_1'], axis=1)
# player_id_col = draft_combine.pop('Player_ID')
# draft_combine.insert(0, 'Player_ID', player_id_col)

In [ ]:
#컬럼 순서 변경
#컬럼 순서 변경
#컬럼 순서 변경
#컬럼 순서 변경



new_order = [ 'Player_ID', 'combine_year', 'Name', 'Body_fat_pct',
       'Hand_Length_inch', 'Hand_width_inch', 'Height_wo_Shoes',
       'Height_w_Shoes', 'Standing_Reach_inch', 'Weight_lbs', 'Wingspan_inch', 
             'LANE_AGILITY_sec', 'SHUTTLE_RUN_sec', 'THREE_QUATER_SPRINT', 'STANDING_VERTICAL_LEAP_inch',
       'MAX_VERTICAL_LEAP_inch', 'MAX_BENCH_PRESS']  # 변경하고 싶은 순서대로 컬럼명을 리스트에 저장
draft_combine = draft_combine[new_order]  # 데이터프레임의 컬럼 순서를 변경

In [ ]:
#불필요한 컬럼 삭제처리
#불필요한 컬럼 삭제처리
#불필요한 컬럼 삭제처리
#불필요한 컬럼 삭제처리




draft_combine = draft_combine.drop(['combine_year'], axis=1)

In [ ]:
#컬럼별 결측치 비율 확인
#컬럼별 결측치 비율 확인
#컬럼별 결측치 비율 확인
#컬럼별 결측치 비율 확인



missing_counts = draft_combine.isnull().sum()
missing_ratios = (missing_counts / len(draft_combine)) * 100
print(missing_ratios)

### Draft combine 동명이인 및 중복 데이터 처리

In [ ]:
# 주요전처리1_draft_combine['Player_ID']변경(일부): 일부 선수의 경우 컴바인 참가 시기와 드래프트 선발 시기가 상이.(자세한 내용은 실험대상 정리파일 참고)
# 주요전처리1_draft_combine['Player_ID']변경(일부): 일부 선수의 경우 컴바인 참가 시기와 드래프트 선발 시기가 상이.(자세한 내용은 실험대상 정리파일 참고)
# 주요전처리1_draft_combine['Player_ID']변경(일부): 일부 선수의 경우 컴바인 참가 시기와 드래프트 선발 시기가 상이.(자세한 내용은 실험대상 정리파일 참고)
# 주요전처리1_draft_combine['Player_ID']변경(일부): 일부 선수의 경우 컴바인 참가 시기와 드래프트 선발 시기가 상이.(자세한 내용은 실험대상 정리파일 참고)
# https://basketball.realgm.com/nba/draft/early_entry/by_year/2013 참조
# https://basketball.realgm.com/nba/draft/early_entry/by_year/2013 참조
# https://basketball.realgm.com/nba/draft/early_entry/by_year/2013 참조
# https://basketball.realgm.com/nba/draft/early_entry/by_year/2013 참조

draft_combine.loc[draft_combine['Player_ID'] == "2005_Dee Brown", 'Player_ID'] = "2006_Dee Brown"
draft_combine.loc[draft_combine['Player_ID'] == "2000_Antonis Fotsis", 'Player_ID'] = "2001_Antonis Fotsis"
draft_combine.loc[draft_combine['Player_ID'] == "2001_Sam Clancy", 'Player_ID'] = "2002_Sam Clancy"
draft_combine.loc[draft_combine['Player_ID'] == "2003_Pavel Podkolzin", 'Player_ID'] = "2004_Pavel Podkolzin"
draft_combine.loc[draft_combine['Player_ID'] == "2004_Nate Robinson", 'Player_ID'] = "2005_Nate Robinson"
draft_combine.loc[draft_combine['Player_ID'] == "2004_Brandon Bass", 'Player_ID'] = "2005_Brandon Bass"
draft_combine.loc[draft_combine['Player_ID'] == "2004_Ryan Gomes", 'Player_ID'] = "2005_Ryan Gomes"
draft_combine.loc[draft_combine['Player_ID'] == "2004_Lawrence Roberts", 'Player_ID'] = "2005_Lawrence Roberts"
draft_combine.loc[draft_combine['Player_ID'] == "2006_Morris Almond", 'Player_ID'] = "2007_Morris Almond"
draft_combine.loc[draft_combine['Player_ID'] == "2008_Robert Vaden", 'Player_ID'] = "2009_Robert Vaden"
draft_combine.loc[draft_combine['Player_ID'] == "2008_Lester Hudson", 'Player_ID'] = "2009_Lester Hudson"
draft_combine.loc[draft_combine['Player_ID'] == "2018_Jaylen Hands", 'Player_ID'] = "2019_Jaylen Hands"
draft_combine.loc[draft_combine['Player_ID'] == "2004_Tiago Splitter", 'Player_ID'] = "2007_Tiago Splitter"
draft_combine.loc[draft_combine['Player_ID'] == "2005_Brandon Rush", 'Player_ID'] = "2008_Brandon Rush"
# draft_combine.loc[draft_combine['Player_ID'] == "2012_Tony Mitchell", 'Player_ID'] = "undrafted_Tony Mitchell"  --> 처음에는 해당 처리를 해줘야 될 줄 알았으나 2012토니미첼(언드래프트티드)와 2013토니미첼(draft)가 구분되므로 해당 처리 취소!! 이상없음을 확인함!


In [ ]:
# 주요전처리2
# 주요전처리2
# 주요전처리2
# 주요전처리2

#2016_Caleb Swanigan, 2017_Caleb Swanigan은 동일한 선수 --> 드래프트된 2017 컴바인 결과로 맵핑하기 위해 2016 데이터는 삭제처리

#2016_Jaron Blossomgame, 2017_Jaron Blossomgame은 동일한 선수 --> 드래프트된 2017 컴바인 결과로 맵핑하기 위해 2016 데이터는 삭제처리

#2016_Malik Newman, 2018_Malik Newman은 동일한 선수 --> 마지막(최근) 컴바인 결과로 맵핑하기 위해 2016 데이터는 삭제처리(드래프트 탈락자이므로 아래에 추가적인 처리 필요)--> 데이터 삭제만 진행함!!!드래프트 탈락자 이지만 아래에 추가적인 처리(드래프트미선발선수에 대한 컴바인 데이터의 id에 "undrafted"명시!-->따라서 3명의 선수에 대한 id변경 처리 취소!!

#2016_Nigel Hayes, 2017_Nigel Hayes은 동일한 선수 --> 마지막(최근) 컴바인 결과로 맵핑하기 위해 2016 데이터는 삭제처리(드래프트 탈락자이므로 아래에 추가적인 처리 필요)--> 데이터 삭제만 진행함!!!드래프트 탈락자 이지만 아래에 추가적인 처리(드래프트미선발선수에 대한 컴바인 데이터의 id에 "undrafted"명시!-->따라서 3명의 선수에 대한 id변경 처리 취소!!

draft_combine = draft_combine[~draft_combine['Player_ID'].isin(["2016_Caleb Swanigan", "2016_Jaron Blossomgame", "2016_Malik Newman", "2016_Nigel Hayes"])]   #데이터 삭제 처리!!!!
#2018_Malik Newman의 경우 드래프트 탈락자이므로 combine에서 생성한 id인 "combine_year + 선수이름"과 df_ncaa_careerlength에서 생성한 id인 "draft_year + 선수이름"과 맵핑이 되지 않음.
#따라서 combine의 id를 수기로 처리: 2018_Malik Newman(draft_combine['Player_ID'] --> undrafted_Malik Newman(df_ncaa_careerlength['Player_ID']으로 대체.
# draft_combine['Player_ID'] = draft_combine['Player_ID'].replace('2018_Malik Newman', 'undrafted_Malik Newman')

######지금 2018_Malik Newman의 케이스를 생각해보면 만약 combine에는 특정연도에 참가하였으나 드래프트에서 탈락하고 나중에 nba에 데뷔한 선수들의 경우는 combine 결과를 맵핑하려면 다른 방법을 생각해야 할 듯.
######예를 들어 combine데이터와 ncaa_careerlength와의 병합과정에서 포함되지 않은 combine데이터 샘플 중 CareerLength_undrafted의 샘플들에 포함되는 샘플을 별도로 추출해서 비교해봐야 할 듯.


#2009_Marcus Thornton(2009drafted/8years/LSU)와 2015_Marcus Thornton(2015drafted/rookie/데뷔x/William&Mary)은 동명이인임.
#현재 2015_Marcus Thornton은 드래프트 리스트(크롤링한 데이터)에 아예 누락되었으므로 추후 확인해야 함.(자세한 내용은 엑셀파일 "실험대상인원정리"참고)
##2009_Marcus Thornton(2009drafted/8years/LSU)은 드래프트 선발 선수이므로 별도 처리 필요없음. 하지만 추후 2015_Marcus Thornton은 확인 필요.


#2017_Nigel Hayes는 드래프트 탈락자이므로 combine에서 생성한 id인 "combine_year + 선수이름"과 df_ncaa_careerlength에서 생성한 id인 "draft_year + 선수이름"과 맵핑이 되지 않음.
#따라서 combine의 id를 수기로 처리: 2017_Nigel Hayes(draft_combine['Player_ID'] --> undrafted_Nigel Hayes(df_ncaa_careerlength['Player_ID']으로 대체.
# draft_combine['Player_ID'] = draft_combine['Player_ID'].replace('2017_Nigel Hayes', 'undrafted_Nigel Hayes')

#2012_Tony Mitchell(undrafted/데뷔(2014.3)/Alabama) 2013_Tony Mitchell(2013drafted/NorthTexas/13-14마지막)은 동명이인이므로 둘 중 한 명(2012_Tony Mitchell을 별도로 처리해서 두 선수를 구분.
##[draft_combine]2012_Tony Mitchell을 TonyMitchell_alabama_undrafted로 변경하고 [df_ncaa_careerlength]Tony Mitchell_alabama를
# draft_combine['Player_ID'] = draft_combine['Player_ID'].replace('2012_Tony Mitchell', 'undrafted_Tony Mitchell')


기존 병합 순서

Stap1. ncaa stat + Careerlength_drafted

Step2. ncaa stat + Careerlength_undrafted

Step3. Step1 + Step2

Stpe4. Step3 + draft combine


수정 후 변경 순서

Step1. Careerlength_drafted + combine

Step2. Careerlength_undrafted + combine

Step3. Step1 + Step2

Step4. Step3 + ncaa stat

### 1.2. data merge(변경)
        1.2.1. Draft_combine + CareerLength(drafted, undrafted)

In [ ]:
#CareerLength_drafted 데이터셋 id생성(Draft_combine와 병합 위한 처리)
#CareerLength_drafted 데이터셋 id생성(Draft_combine와 병합 위한 처리)
#CareerLength_drafted 데이터셋 id생성(Draft_combine와 병합 위한 처리)
#CareerLength_drafted 데이터셋 id생성(Draft_combine와 병합 위한 처리)

CareerLength_drafted['Draft_year'] = CareerLength_drafted['Draft_year'].astype(str)
CareerLength_drafted['Player_ID_new'] = CareerLength_drafted['Draft_year'] + '_' + CareerLength_drafted['Name']


# draft_combine['Player_ID'] = draft_combine.apply(lambda row: row['Player_ID_1'] + ' _None' if pd.isna(row['College_ncaa_1']) else row['Player_ID_1'], axis=1)
# player_id_col = draft_combine.pop('Player_ID')
# draft_combine.insert(0, 'Player_ID', player_id_col)

In [ ]:
comb_career_draft = pd.merge(CareerLength_drafted, draft_combine, left_on='Player_ID_new', right_on='Player_ID', how='left')

In [ ]:
#CareerLength_drafted + draft_combine 병합 과정에서 탈락된 draft_combine 샘플들의 ID의 year부분을 모두 undrafted로 변경
#CareerLength_drafted + draft_combine 병합 과정에서 탈락된 draft_combine 샘플들의 ID의 year부분을 모두 undrafted로 변경
#CareerLength_drafted + draft_combine 병합 과정에서 탈락된 draft_combine 샘플들의 ID의 year부분을 모두 undrafted로 변경
#CareerLength_drafted + draft_combine 병합 과정에서 탈락된 draft_combine 샘플들의 ID의 year부분을 모두 undrafted로 변경


# CareerLength_drafted와 draft_combine을 모두 포함하는 데이터프레임 생성
Comb_career_draft_sum = pd.merge(CareerLength_drafted, draft_combine, left_on='Player_ID_new', right_on='Player_ID', how='outer')

# 전 단계의 병합과정에서 탈락된 즉, draft_combine에만 포함되는 샘플들 추출(combine_undrafted: 컴바인 참가했으나 드래프트 미선발인원 추출)
combine_undrafted = Comb_career_draft_sum[Comb_career_draft_sum['Player_ID'].notna() & Comb_career_draft_sum['Player_ID_new'].isna()]


#combine_undrafted을 draft_combine과 동일하게 컬럼 수정
# 'Name_y' 컬럼을 'Name'으로 변경
combine_undrafted = combine_undrafted.rename(columns={'Name_y': 'Name'})

# draft_combine의 컬럼 목록을 가져옵니다.
draft_combine_columns = draft_combine.columns

# combine_undrafted의 컬럼을 draft_combine과 동일한 순서로 정렬하고, 불필요한 컬럼을 제거합니다.
combine_undrafted = combine_undrafted[draft_combine_columns]


# combine_undrafted(=컴바인 참가했으나 드래프트 미선발인원 추출)의 Player_ID의 앞부분은 combine 참가 연도이나 드래프트 미선발 인원과의 병합을 위해 "undrafted"로 변경 처
combine_undrafted['Player_ID_new'] = combine_undrafted['Player_ID'].apply(lambda x: 'undrafted_' + x.split('_')[1])

#CareerLength_undrafted와 draft_combine병합을 위해 ID 생성 작업

#CareerLength_undrafted["Draft_year"]가 null값이므로 "undrafted"로 채워줌
CareerLength_undrafted['Draft_year'] = 'undrafted'

##Player_ID_new 컬럼 생성
CareerLength_undrafted['Player_ID_new'] = CareerLength_undrafted['Draft_year'] + '_' + CareerLength_undrafted['Name']





In [ ]:
comb_career_undrafted = pd.merge(CareerLength_undrafted, combine_undrafted, left_on='Player_ID_new', right_on='Player_ID_new', how='left')

In [ ]:
comb_career = pd.concat([comb_career_draft, comb_career_undrafted], axis=0, ignore_index=True)
print(comb_career.head())

### 1.2. data merge(변경)
        1.2.2. comb_career + ncaa_stat

In [ ]:
# #ncaa_stat_new + comb_career 데이터셋을 선수id와 학교명을 key값으로 병합. 
# #ncaa_stat_new + comb_career 데이터셋을 선수id와 학교명을 key값으로 병합. 
# #ncaa_stat_new + comb_career 데이터셋을 선수id와 학교명을 key값으로 병합. 
# #ncaa_stat_new + comb_career 데이터셋을 선수id와 학교명을 key값으로 병합. 





# Career length의 "Name"과 "College_ncaa_1"의 값을 기준으로 Ncaa_stat의 "Player", "School"이 일치하는 데이터만 추출
ncaa_comb_caeeer_1 = pd.merge(ncaa_stat, comb_career, left_on=["Player_ID", "School"], right_on=["Player_ID_1", "College_ncaa_1"], how='inner')

# Career length의 "Name"과 "College_ncaa_1"의 값을 기준으로 Ncaa_stat의 "Player", "School"이 일치하는 데이터만 추출
ncaa_comb_caeeer_2 = pd.merge(ncaa_stat, comb_career, left_on=["Player_ID", "School"], right_on=["Player_ID_2", "College_ncaa_2"], how='inner')

# Career length의 "Name"과 "College_ncaa_1"의 값을 기준으로 Ncaa_stat의 "Player", "School"이 일치하는 데이터만 추출
ncaa_comb_caeeer_3 = pd.merge(ncaa_stat, comb_career, left_on=["Player_ID", "School"], right_on=["Player_ID_3", "College_ncaa_3"], how='inner')

# Career length의 "Name"과 "College_ncaa_1"의 값을 기준으로 Ncaa_stat의 "Player", "School"이 일치하는 데이터만 추출
ncaa_comb_caeeer_4 = pd.merge(ncaa_stat, comb_career, left_on=["Player_ID", "School"], right_on=["Player_ID_4", "College_ncaa_4"], how='inner')




In [ ]:
# 데이터프레임 병합
# 데이터프레임 병합
# 데이터프레임 병합
# 데이터프레임 병합

ncaa_comb_career = pd.concat([ncaa_comb_caeeer_1, ncaa_comb_caeeer_2, ncaa_comb_caeeer_3, ncaa_comb_caeeer_4], axis=0)

# 인덱스 리셋
# 인덱스 리셋
# 인덱스 리셋
# 인덱스 리셋


ncaa_comb_career.reset_index(drop=True, inplace=True)


In [ ]:
import hashlib

# 해시 함수를 적용하여 고유 식별자를 생성하는 함수
def generate_hash_id(row):
    # 필요한 컬럼 값을 결합하여 문자열 생성
    id_string = f"{row['Player_ID_new']}_{row['College_ncaa_1']}_{row['College_ncaa_2']}_{row['College_ncaa_3']}_{row['College_ncaa_4']}_{row['Draft_year']}_{row['Draft_team']}_{row['Draft_overall']}_{row['Debut']}_{row['Experience']}"
    # 해시 함수를 사용하여 고유 식별자 생성
    return hashlib.md5(id_string.encode()).hexdigest()

# 고유 식별자 컬럼 생성
ncaa_comb_career['ID'] = ncaa_comb_career.apply(generate_hash_id, axis=1)

# 결과 확인
print(ncaa_comb_career[['Player_ID_new', 'ID']].head())


In [ ]:
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.

# Player_ID_x와 Season 컬럼의 값이 같은 중복된 샘플을 확인
duplicate_samples = ncaa_comb_career[ncaa_comb_career.duplicated(['Player_ID_x', 'Class'], keep=False)]

# 중복된 샘플 확인
print(duplicate_samples)


In [ ]:
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.

# Player_ID_x와 Season 컬럼의 값이 같은 중복된 샘플을 확인
duplicate_samples = ncaa_comb_career[ncaa_comb_career.duplicated(['Player_ID_x', 'Season'], keep=False)]

# 중복된 샘플 확인
print(duplicate_samples)


In [ ]:
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.
## Player_ID_x컬럼과 Season컬럼 기준으로 중복된 값이 있는지 검사 수행 및 중복값 삭제 처리.

# Jaylen Hands_ucla와 2017-18인 샘플의 갯수를 구하기
sample_count = ncaa_comb_career[(ncaa_comb_career['Player_ID_x'] == 'Jaylen Hands_ucla') & (ncaa_comb_career['Season'] == '2017-18')].shape[0]
print("Number of samples:", sample_count)

# Jaylen Hands_ucla와 2018-19인 샘플의 갯수를 구하기
sample_count = ncaa_comb_career[(ncaa_comb_career['Player_ID_x'] == 'Jaylen Hands_ucla') & (ncaa_comb_career['Season'] == '2018-19')].shape[0]
print("Number of samples:", sample_count)

# 해당 샘플 중 하나를 삭제
ncaa_comb_career = ncaa_comb_career[~((ncaa_comb_career['Player_ID_x'] == 'Jaylen Hands_ucla') & (ncaa_comb_career['Season'] == '2017-18')) | ~ncaa_comb_career.duplicated(subset=['Player_ID_x', 'Season'])]
# 해당 샘플 중 하나를 삭제
ncaa_comb_career = ncaa_comb_career[~((ncaa_comb_career['Player_ID_x'] == 'Jaylen Hands_ucla') & (ncaa_comb_career['Season'] == '2018-19')) | ~ncaa_comb_career.duplicated(subset=['Player_ID_x', 'Season'])]

# 샘플 삭제 후 확인
sample_count_after_deletion = ncaa_comb_career[(ncaa_comb_career['Player_ID_x'] == 'Jaylen Hands_ucla') & (ncaa_comb_career['Season'] == '2017-18')].shape[0]
print("Number of samples after deletion:", sample_count_after_deletion)
# 샘플 삭제 후 확인
sample_count_after_deletion = ncaa_comb_career[(ncaa_comb_career['Player_ID_x'] == 'Jaylen Hands_ucla') & (ncaa_comb_career['Season'] == '2018-19')].shape[0]
print("Number of samples after deletion:", sample_count_after_deletion)


In [ ]:
# 컬럼명 변경
# 컬럼명 변경
# 컬럼명 변경
# 컬럼명 변경

ncaa_comb_career.rename(columns={'Name_x': 'Name'}, inplace=True)
ncaa_comb_career.rename(columns={'Player_ID_new': 'Player_ID'}, inplace=True)
# # 결과 확인
# print(CareerLength_undrafted.head())


In [ ]:
# #불필요한 컬럼 삭제처리
# #불필요한 컬럼 삭제처리
# #불필요한 컬럼 삭제처리
# #불필요한 컬럼 삭제처리


ncaa_comb_career = ncaa_comb_career.drop(['Player_ID_x', 'Player', 'Height_inch', 'Weight', 'Player_ID_1', 'Player_ID_2', 'Player_ID_3', 'Player_ID_4', 'Position', 'Born', 'Birth', 'Player_ID_y', 'Name_y', 
                                          "Weight_lbs", 'ORB_totals', 'DRB_totals'], axis=1)
# #컬럼 삭제 처리: ORB, DRB(공격 및 수비리바운드는 결측치 비율이 높으므로 총 리바운드 피처만 사용할 것)


In [ ]:
# 데이터 전처리
# 데이터 전처리
# 데이터 전처리
# 데이터 전처리

# 'Class' 및 'Pos' 컬럼 전처리

class_mapping = {'FR': 1, 'SO': 2, 'JR': 3, 'SR': 4}
pos_mapping = {'G': 1, 'F': 3, 'C': 5}
ncaa_comb_career['Class'] = ncaa_comb_career['Class'].map(class_mapping)
ncaa_comb_career['Pos'] = ncaa_comb_career['Pos'].map(pos_mapping)

In [ ]:
#merged_all_comb 컬럼순서 조정
#merged_all_comb 컬럼순서 조정
#merged_all_comb 컬럼순서 조정
#merged_all_comb 컬럼순서 조정



new_order = ["ID", "Player_ID", "Name", "Season", "School", "Class", 'Pos',
             "Height_cm", "Weight_kg", 'College_ncaa_1', 'College_ncaa_2', 'College_ncaa_3', 'College_ncaa_4', "Recruiting Rank", "Draft_year", "Draft_overall", "Draft_team", "Debut", "Debut_yrs", "Age", 
             
             
              "Body_fat_pct", "Hand_Length_inch", "Hand_width_inch",
             "Height_wo_Shoes", "Height_w_Shoes", "Standing_Reach_inch", "Wingspan_inch","LANE_AGILITY_sec", "SHUTTLE_RUN_sec", "THREE_QUATER_SPRINT", "STANDING_VERTICAL_LEAP_inch",
             "MAX_VERTICAL_LEAP_inch", "MAX_BENCH_PRESS", 
             
             
             "G_totals", "MP_totals", "FG_totals", "FGA_totals", "FG%_totals",
             "2P_totals", "2PA_totals", "2P%_totals", "3P_totals", "3PA_totals", "3P%_totals", 
             "FT_totals", "FTA_totals", "FT%_totals", "TRB_totals",
             "AST_totals", "STL_totals", "BLK_totals", "TOV_totals", "PF_totals", "PTS_totals",
             "FG_per_min", "FGA_per_min", "2P_per_min", "2PA_per_min", "3P_per_min", "3PA_per_min",
             "FT_per_min", "FTA_per_min", "TRB_per_min", "AST_per_min", "STL_per_min", "BLK_per_min",
             "TOV_per_min", "PF_per_min", "PTS_per_min",
             "TS%_advanced", "eFG%_advanced", "3PAr_advanced", "FTr_advanced", "PProd_advanced",
             "ORB%_advanced", "DRB%_advanced", "TRB%_advanced", "AST%_advanced", "OBPM_advanced",
             "DBPM_advanced", "BPM_advanced", "STL%_advanced", "BLK%_advanced", "TOV%_advanced",
             "USG%_advanced", "OWS_advanced", "DWS_advanced", "WS_advanced", "WS/40_advanced", "PER_advanced",
             "FG_per_poss", "FGA_per_poss", "2P_per_poss", "2PA_per_poss", "3P_per_poss",
             "3PA_per_poss", "FT_per_poss", "FTA_per_poss", "TRB_per_poss", "AST_per_poss", "STL_per_poss",
             "BLK_per_poss", "TOV_per_poss", "PF_per_poss", "PTS_per_poss", "ORtg_per_poss", "DRtg_per_poss",
            
             "Experience"]  # 변경하고 싶은 순서대로 컬럼명을 리스트에 저장
ncaa_comb_career = ncaa_comb_career[new_order]  # 데이터프레임의 컬럼 순서를 변경